In [2]:
import pandas as pd
import numpy as np

import time, warnings
import datetime as dt

import sklearn.cluster as cluster
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture

from sklearn.metrics import silhouette_samples, silhouette_score
from scipy.spatial.distance import cdist

import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
%matplotlib inline
import seaborn as sns

warnings.filterwarnings("ignore")

In [3]:
# Causal impact imports
from IPython.core.pylabtools import figsize
figsize(14, 6)
import statsmodels as sm
from statsmodels.tsa.statespace.structural import UnobservedComponents
from statsmodels.tsa.arima_process import ArmaProcess
from causalimpact import CausalImpact

In [96]:
df = pd.read_excel("Online_Retail.xlsx")
test = pd.read_excel("onlineretail.xlsx")

In [97]:
df.shape, test.shape

((541909, 8), (541909, 8))

In [99]:
df2 = df.groupby(['CustomerID','Country']).agg(['count'])
test2 = test.groupby(['CustomerID','Country']).agg(['count'])

In [39]:
df.groupby(['CustomerID','Country']).agg(['count'])

,,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice
,,count,count,count,count,count,count
CustomerID,Country,,,,,,
12346.0,United Kingdom,2,2,2,2,2,2
12347.0,Iceland,182,182,182,182,182,182
12348.0,Finland,31,31,31,31,31,31
12349.0,Italy,73,73,73,73,73,73
12350.0,Norway,17,17,17,17,17,17
...,...,...,...,...,...,...,...
18280.0,United Kingdom,10,10,10,10,10,10
18281.0,United Kingdom,7,7,7,7,7,7


In [5]:
df3 = df2.reset_index()

In [6]:
df3 = df3[['CustomerID','Country']]

In [20]:
df3.CustomerID.nunique()

4372

In [28]:
df3

,CustomerID,Country
,,
0,12346.0,United Kingdom
1,12347.0,Iceland
2,12348.0,Finland
3,12349.0,Italy
4,12350.0,Norway
...,...,...
4375,18280.0,United Kingdom
4376,18281.0,United Kingdom
4377,18282.0,United Kingdom


In [58]:
# Customer table
#df3.to_csv('CustomerTable.csv')


In [42]:
# Transaction Table
dft = df.groupby(['InvoiceNo','InvoiceDate','CustomerID','Country','StockCode','Quantity']).agg(['count'])

In [43]:
dft1 = dft.reset_index()

In [44]:
# final transaction table
dft2 = dft1[['InvoiceNo','InvoiceDate','CustomerID','Country','StockCode','Quantity']]

In [45]:
dft2

,InvoiceNo,InvoiceDate,CustomerID,Country,StockCode,Quantity
,,,,,,
0,536365,2010-12-01 08:26:00,17850.0,United Kingdom,21730,6
1,536365,2010-12-01 08:26:00,17850.0,United Kingdom,22752,2
2,536365,2010-12-01 08:26:00,17850.0,United Kingdom,71053,6
3,536365,2010-12-01 08:26:00,17850.0,United Kingdom,84029E,6
4,536365,2010-12-01 08:26:00,17850.0,United Kingdom,84029G,6
...,...,...,...,...,...,...
401544,C581490,2011-12-09 09:57:00,14397.0,United Kingdom,23144,-11
401545,C581499,2011-12-09 10:28:00,15498.0,United Kingdom,M,-1
401546,C581568,2011-12-09 11:57:00,15311.0,United Kingdom,21258,-5


In [46]:
dft2.to_csv('TransactionsTable.csv')

In [108]:
# Item table
dfI = df.groupby(['StockCode','Description','UnitPrice']).agg(['count'])

dfI1 = dfI.reset_index()

dfI2 = dfI1[['StockCode','Description','UnitPrice']]

In [126]:
testI = test.groupby(['StockCode','Description']).agg({'UnitPrice':np.mean})
testI1 = testI.reset_index()
testI2 = testI1[['StockCode','Description','UnitPrice']]

In [125]:
testI2.to_csv('ItemsTable.csv')

In [109]:
dfI2.head()

,StockCode,Description,UnitPrice
,,,
0,10002,INFLATABLE POLITICAL GLOBE,0.85
1,10002,INFLATABLE POLITICAL GLOBE,1.63
2,10002,INFLATABLE POLITICAL GLOBE,1.66
3,10080,GROOVY CACTUS INFLATABLE,0.39
4,10080,GROOVY CACTUS INFLATABLE,0.85


In [117]:
testI2.head()

,StockCode,Description,UnitPrice
0,10002,INFLATABLE POLITICAL GLOBE,1.086620
1,10080,GROOVY CACTUS INFLATABLE,0.410909
2,10080,check,0.000000
3,10120,DOGGY RUBBER,0.210000
4,10125,MINI FUNKY DESIGN TAPES,0.859681


In [81]:
#dfI2.to_csv('ItemsTable.csv')

In [82]:
dfI2

,StockCode,Description,CustomerID,Country,UnitPrice
,,,,,
0,10002,INFLATABLE POLITICAL GLOBE,12451.0,Switzerland,0.85
1,10002,INFLATABLE POLITICAL GLOBE,12510.0,Spain,0.85
2,10002,INFLATABLE POLITICAL GLOBE,12583.0,France,0.85
3,10002,INFLATABLE POLITICAL GLOBE,12637.0,France,0.85
4,10002,INFLATABLE POLITICAL GLOBE,12673.0,Germany,0.85
...,...,...,...,...,...
277412,POST,POSTAGE,17935.0,United Kingdom,2.90
277413,POST,POSTAGE,17940.0,United Kingdom,75.00
277414,POST,POSTAGE,18223.0,United Kingdom,2.84


In [83]:
len(df)

541909

In [137]:
dfs1 = dft2.merge(df3, on=['CustomerID','Country'], how='left')

In [138]:
dfs1

,InvoiceNo,InvoiceDate,CustomerID,Country,StockCode,Quantity
,,,,,,
0,536365,2010-12-01 08:26:00,17850.0,United Kingdom,21730,6
1,536365,2010-12-01 08:26:00,17850.0,United Kingdom,22752,2
2,536365,2010-12-01 08:26:00,17850.0,United Kingdom,71053,6
3,536365,2010-12-01 08:26:00,17850.0,United Kingdom,84029E,6
4,536365,2010-12-01 08:26:00,17850.0,United Kingdom,84029G,6
...,...,...,...,...,...,...
401544,C581490,2011-12-09 09:57:00,14397.0,United Kingdom,23144,-11
401545,C581499,2011-12-09 10:28:00,15498.0,United Kingdom,M,-1
401546,C581568,2011-12-09 11:57:00,15311.0,United Kingdom,21258,-5


In [139]:
dfs2 = dfs1.merge(testI2, on=['StockCode'], how='left')

In [141]:
dfs2

,StockCode,"(InvoiceNo, )","(InvoiceDate, )","(CustomerID, )","(Country, )","(StockCode, )","(Quantity, )",Description,UnitPrice
0,21730,536365,2010-12-01 08:26:00,17850.0,United Kingdom,21730,6,GLASS STAR FROSTED T-LIGHT HOLDER,5.322837
1,22752,536365,2010-12-01 08:26:00,17850.0,United Kingdom,22752,2,SET 7 BABUSHKA NESTING BOXES,9.767352
2,71053,536365,2010-12-01 08:26:00,17850.0,United Kingdom,71053,6,WHITE METAL LANTERN,4.627866
3,71053,536365,2010-12-01 08:26:00,17850.0,United Kingdom,71053,6,WHITE MOROCCAN METAL LANTERN,6.595185
4,84029E,536365,2010-12-01 08:26:00,17850.0,United Kingdom,84029E,6,RED WOOLLY HOTTIE WHITE HEART.,5.102071
...,...,...,...,...,...,...,...,...,...
532976,M,C581499,2011-12-09 10:28:00,15498.0,United Kingdom,M,-1,Manual,375.566392
532977,21258,C581568,2011-12-09 11:57:00,15311.0,United Kingdom,21258,-5,?,0.000000
532978,21258,C581568,2011-12-09 11:57:00,15311.0,United Kingdom,21258,-5,VICTORIAN SEWING BOX LARGE,16.494451
532979,20979,C581569,2011-12-09 11:58:00,17315.0,United Kingdom,20979,-5,36 PENCILS TUBE RED RETROSPOT,1.569101


In [134]:
dfs2.groupby(['CustomerID','Country']).agg(count)

KeyError: 'CustomerID'

In [130]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
